In [62]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
from proj1_helpers import *
from data_helpers import *
from cross_validation import *
from plots import *

In [99]:
#LOAD TRAINING DATA
DATA_TRAIN_PATH = 'data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [100]:
#CLEAN TRAINING DATA
bounds = [0.0, 0.5]
tx_clean, ind_remov = treat_undefined_values(bounds, tX)
ind_remov

[4, 5, 6, 12, 26, 27, 28]

In [101]:
#MODEL BUILDING
tx, mean, std = standardize(tx_clean,0)
tx_augmented = build_poly_all_features(tx,4)
y,tx = build_model_data(tx_augmented,y)
#y = classify(y)
num_samples = len(y)
num_features = tx.shape[1]
y = y.reshape(num_samples,1)


In [102]:
num_samples, num_features
tx.shape

(250000, 93)

In [103]:
from compute_gradient import *
from cost import *
from implementations import *

In [104]:
#Step-size

lambdas = np.logspace (-20,1,22)


#Regularization factor
# lambda_ = 0.01 142000 loss 10 000 iters
#lambda_ = 0.001 141000 loss 10 000 iters
#lambda_ = 0.02
#loss_min = np.inf
tx.shape,y.shape,y

((250000, 93), (250000, 1), array([[ 1.],
        [-1.],
        [-1.],
        ...,
        [ 1.],
        [-1.],
        [-1.]]))

In [105]:
losses_tr = np.zeros(len(lambdas))
losses_val = np.zeros(len(lambdas))

k_fold = 10
seed = 1
loss_min = np.inf
max_accuracy = 0

#CHOOSE LAMBDA AND GAMMA
for ind_lambda, lambda_ in enumerate(lambdas):
    
    loss_training = 0
    loss_validation = 0
    training_accuracy = 0
    validation_accuracy = 0
    for i,k in enumerate(range(k_fold)):
        tx_train, y_train, tx_val, y_val = cross_validation(y, tx, k, k_fold, seed)
        w, loss_train_k = ridge_regression(y_train, tx_train, lambda_)
        loss_val_k = ridge_loss (y_val,tx_val,w,lambda_)
        loss_training += loss_train_k
        loss_validation += loss_val_k
        training_accuracy += predict_accuracy(y_train,tx_train,w, 'linear')
        validation_accuracy += predict_accuracy(y_val,tx_val,w, 'linear')
        
    training_accuracy /= k_fold
    validation_accuracy /= k_fold
    loss_training /= k_fold
    loss_validation /= k_fold
    print(lambda_)
    print(loss_training)
    print(loss_validation)
    print(training_accuracy)
    print(validation_accuracy)
    if (validation_accuracy > max_accuracy):
        weights_star = w
        lambda_star = lambda_
        loss_min = loss_validation
        max_accuracy = validation_accuracy 
    losses_tr[ind_lambda] = loss_training
    losses_val[ind_lambda] = loss_validation



linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
1e-20
0.3134610494490199
3.622827883167436
77.61955555555555
77.60119999999999
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
1e-19
0.31346104946993797
3.6228278831883545
77.61955555555555
77.60119999999999
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
1e-18
0.3134610496791179
3.622827883397534
77.61955555555555
77.60119999999999
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
1e-17
0.31346105177091765
3.6228278854893325
77.61955555555555
77.60119999999999
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
linear
l

In [ ]:
lambdas

In [106]:
loss_min, lambda_star, max_accuracy

(2.945208588852501, 1e-09, 77.6112)

In [107]:
#PREDICT TRAINING ACCURACY
training_accuracy = predict_accuracy(y,tx,weights_star,'linear')
training_accuracy

linear


77.6108

In [108]:
#LOAD TEST SET
DATA_TEST_PATH = 'data/test.csv'
ytest, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [109]:
#CLEAN AND STANDARDIZE TEST SET
tX_test_clean = np.delete(tX_test, ind_remov, axis=1)
tX_test_clean.shape


(568238, 23)

In [110]:
tx_test_clean = remove_undefined_values (tX_test_clean)
tx_test_clean.shape

(568238, 23)

In [111]:
tx_test_clean, _, _ = standardize(tx_test_clean,0)


In [112]:
tx_test_augmented = build_poly_all_features(tx_test_clean,4)

In [113]:
tx_test_augmented.shape

(568238, 92)

In [114]:
#BUILD TEST MODEL
ytest, tx_test = build_model_data(tx_test_augmented,ytest)

In [115]:
#PREDICT LABELS
OUTPUT_PATH = 'data/submission_reg_logistic-degre4.csv'
y_pred = predict_labels(weights_star, tx_test,'linear')
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

linear


In [92]:
y

array([[ 1.],
       [-1.],
       [-1.],
       ...,
       [ 1.],
       [-1.],
       [-1.]])

In [93]:
y_pred

array([[ 1.],
       [ 1.],
       [ 1.],
       ...,
       [ 1.],
       [-1.],
       [ 1.]])

In [95]:
ytest

array([1., 1., 1., ..., 1., 1., 1.])